In [1]:
%classpath add jar ../../konduit.jar

## Model Link
The dl4j model is taken from the dl4j-example here: https://github.com/eclipse/deeplearning4j-examples/blob/master/mvn-project-template/src/main/java/org/deeplearning4j/examples/sample/LeNetMNIST.java

In [2]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.pipeline.api.data.Data;
import ai.konduit.serving.pipeline.api.data.Image;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;
import ai.konduit.serving.pipeline.impl.format.JavaImageFactory;
import ai.konduit.serving.pipeline.registry.ImageFactoryRegistry;
import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.util.ObjectMappers;
import ai.konduit.serving.pipeline.registry.NDArrayConverterRegistry;
import ai.konduit.serving.data.nd4j.format.ND4JConverters;

public class OCREndPoint implements Endpoint {

    private PipelineExecutor pipelineExecutor;

    public OCREndPoint(PipelineExecutor pipelineExecutor) { 
        this.pipelineExecutor = pipelineExecutor; 
        ImageFactoryRegistry.addFactory(new JavaImageFactory()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.Nd4jToSerializedConverter()); 
        NDArrayConverterRegistry.addConverter(new ND4JConverters.SerializedToNd4jArrConverter());
    }

    public HttpMethod type() { return HttpMethod.POST; }

    public String path() { return "/infer"; }

    public List<String> consumes() { return Arrays.asList("application/octet-stream","multipart/form-data"); }

    public List<String> produces() { return Arrays.asList("application/json"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> {
            handler.vertx().executeBlocking(taskHandler -> {
                Data image = Data.empty();
                
                try {
                    image.put("image",Image.create(ImageIO.read(new File(handler.fileUploads().iterator().next().uploadedFileName()))));
                } catch (IOException e) {
                    e.printStackTrace();
                }

                Data exec = pipelineExecutor.exec(image);
                
                handler.response().end(ObjectMappers.toJson(exec.getNDArray("layer5").getAs(float[].class)));
                taskHandler.complete();
            },resultHandler -> {
                if(resultHandler.failed()) {
                    if(resultHandler.cause() != null)
                        if(handler.vertx().exceptionHandler() != null)
                            handler.vertx().exceptionHandler().handle(resultHandler.cause());
                        else {
                            resultHandler.cause().printStackTrace();
                        }
                    else {
                        System.err.println("Failed to process classification endpoint async task. Unknown cause.");
                    }
                }
            });

        };
    }
}

ai.konduit.OCREndPoint

In [3]:
package ai.konduit;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class OCREndPoints implements HttpEndpoints {

    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new OCREndPoint(pipelineExecutor));
    }
}

ai.konduit.OCREndPoints

In [4]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("Saved %s at: %s%n", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker2956758940093063187/outDir
/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar
Saved /tmp/beaker2956758940093063187/outDir:/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar at: /home/shams/PycharmProjects/konduit-serving-demo/demos/5-dl4j-mnist/classpath


null

In [5]:
%%bash
java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id dl4j-mnist -c dl4j.json -rwm -b

Starting konduit server...
Using classpath: /tmp/beaker2956758940093063187/outDir:/home/shams/PycharmProjects/konduit-serving-demo/konduit.jar
INFO: Running command /home/shams/miniconda3/envs/cuda11/jre/bin/java -Dkonduit.logs.file.path=/home/shams/.konduit-serving/command_logs/dl4j-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_6df2c1386a194a73.xml ai.konduit.serving.cli.launcher.KonduitServingLauncher run --instances 1 -s inference -c dl4j.json -Dserving.id=dl4j-mnist
For server status, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher list'
For logs, execute: 'java ai.konduit.serving.cli.launcher.KonduitServingLauncher logs dl4j-mnist'



In [8]:
%%bash
konduit logs dl4j-mnist -l 100

21:55:23.322 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /home/shams/PycharmProjects/konduit-serving-demo/demos/5-dl4j-mnist/dl4j.json
21:55:23.327 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
21:55:23.329 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
21:55:23.329 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'dl4j-mnist'
21:55:23.582 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 28 PipelineStepRunnerFactory instances
21:55:23.784 [vert.x-worker-thread-0] INFO  org.nd4j.linalg.factory.Nd4jBackend - Loaded [JCublasBackend] backend
21:55:23.785 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.nd4j.linalg.jblas.JblasBackend] of provided class-loader.
21:55:23.785 [vert.x-worker-thread-0] ERROR o.n.comm

In [9]:
%%bash
konduit list


Listing konduit servers...

No konduit servers found.



In [1]:
%%html
<img src="test-image.jpg"/>

In [27]:
%%bash
curl -s -H "Content-Type: multipart/form-data" -X POST -F "image=@test-image.jpg" http://localhost:$(konduit inspect dl4j-mnist -q {port})/infer

[ 1.845163E-5, 1.8346094E-6, 0.31436875, 0.43937472, 2.6101702E-8, 0.24587035, 5.9430695E-6, 3.3270408E-4, 6.3698195E-8, 2.708706E-5 ]



In [28]:
%%bash
konduit stop dl4j-mnist

Stopping konduit server 'dl4j-mnist'
Application 'dl4j-mnist' terminated with status 0

